# parcels regions

runs parcels on existing netcdf files

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import utils
from parcels_utils import HFRGrid, read_netcdf_info
from parcels_sim import ParcelsSimulation
from parcels_analysis import add_feature_set_to_result

## configuration

change the contents of `configs` for the simulation configuration you want

In [ ]:
configs = [
    "plume_track.json",
#     "plume_track_totsdlj.json",
#     "tijuana_interped.json",
#     "tijuana_lin.json",
#     "tijuana_lin_aggr.json",
#     "tijuana_less.json",
#     "tijuana_now.json",
#     "tijuana_onerep.json",
#     "tijuana_range.json",
#     "buoy_track.json",
]

loaded_configs = [utils.load_config(utils.PARCELS_CONFIGS_DIR / path) for path in configs]
grids = [HFRGrid(read_netcdf_info(cfg["netcdf_path"])) for cfg in loaded_configs]
sims = []
for i in range(len(loaded_configs)):
    sim = ParcelsSimulation(loaded_configs[i]["name"], grids[i], loaded_configs[i]["parcels_config"])
    sims.append(sim)

## Animated gif stuff and particle simulation

### ParticleSet and spawn point setup

note about interpolation methods: only `linear` works if you want to use the FieldSet in a ParticleSet.

In [ ]:
[sim.pset.show(field="vector") for sim in sims]

### simulation execution

In [ ]:
for sim in sims:
    sim.execute()

### plot and gif generation

gif generation requires [magick](https://imagemagick.org/index.php)

if it doesn't work just comment out the gif generation line

the gifs will be saved `snapshots/`

In [ ]:
for i, sim in enumerate(sims):
    cfg = loaded_configs[i]
    if cfg["save_snapshots"]:
        plotting_cfg = cfg["plotting_config"]
        if "coastline" in plotting_cfg and plotting_cfg["coastline"] not in (None, {}):
            # coastline stuff, use to delete particles that collide with a coastline
            lats, lons = utils.load_geo_points(plotting_cfg["coastline"])
            sim.parcels_result.add_coastline(lats, lons)
            sim.parcels_result.process_coastline_collisions()
            print("processed collisions")
        if "plot_feature_set" in plotting_cfg and plotting_cfg["plot_feature_set"] not in (None, ""):
            add_feature_set_to_result(sim.parcels_result, plotting_cfg["plot_feature_set"])
        sim.parcels_result.generate_all_plots(os.path.join(plotting_cfg["save_dir_snapshots"], sim.name), domain=plotting_cfg["shown_domain"], land=plotting_cfg["draw_coasts"])
        # comment out the generate_gif line if it doesn't work
        print(sim.parcels_result.generate_gif(os.path.join(plotting_cfg["save_dir_snapshots"], f"{sim.name}.gif")))